# Evaluation of methods
- Contains evaluations of: Segmentations, postprocessing order removal, postprocessing cross-reference linking

In [2]:
import json_helpers as jh
from scraping_and_segmenting_helpers import *

E1_SEGMENTER_STATS = f"{SEGMENTER_STATS_FOLDER}stats_segmenter_e1.txt"
E2_SEGMENTER_STATS = f"{SEGMENTER_STATS_FOLDER}stats_segmenter_e2.txt"

E1_EVAL_ORDER = 'evaluations/order_jsons/e1_test_order'
E2_EVAL_ORDER = 'evaluations/order_jsons/e2_test_order'

E1_ORDER_STATS = 'evaluations/order_stats/e1_order_removal_precision.txt'
E2_ORDER_STATS = 'evaluations/order_stats/e2_order_removal_precision.txt'

E1_TEST_LINKS = 'evaluations/links_jsons/e1_test_links'
E2_TEST_LINKS = 'evaluations/links_jsons/e2_test_links'

E1_STATS_LINKS = 'evaluations/links_stats/e1_links_precision.txt'
E2_STATS_LINKS = 'evaluations/links_stats/e2_links_precision.txt'

E1_STATS_LINKS_JSON = 'evaluations/links_stats/e1_links_recall'
E2_STATS_LINKS_JSON = 'evaluations/links_stats/e2_links_recall'

E1_TEST_LINKS_JSON = 'evaluations/links_jsons/e1_test_linksX'
E2_TEST_LINKS_JSON = 'evaluations/links_jsons/e2_test_linksX'


### Evaluation functions

In [3]:
def recall_segmentation(entries: list[dict]):
    nr_entries = len(entries)
    true_positives = sum([1 if (entry['entryid'] != "na") else 0 for entry in entries])
    recall = true_positives / nr_entries
    return recall

def write_recall_to_stats(filename: str, recall: float):
    with open(filename, 'a') as file:
        file.write("------------\n")
        file.write(f"Recall: {recall}\n")

def precision(entries: list[dict], key: str) -> float:
    true_positives = sum([entry[key] for entry in entries])
    return true_positives / len(entries)

def write_to_stats(precision: float, desc: str, eval_file: str):
    with open(eval_file, "w", encoding='utf-8') as file:
        file.write(f"{desc}: {precision}")

## Evaluation of segmentations
* Uses the manually annotated test data.
* Writes the segmentation recall (true sample articles / number of these found in segmentations) to stats.

In [4]:
# Retrieve test json items
e1_test_entries = jh.read_items(f"{TEST_SEGMENTER_FOLDER}e1_test_segmenter")
e2_test_entries = jh.read_items(f"{TEST_SEGMENTER_FOLDER}e2_test_segmenter")

In [5]:
e1_recall = recall_segmentation(e1_test_entries)
e2_recall = recall_segmentation(e2_test_entries)

write_recall_to_stats(E1_SEGMENTER_STATS, e1_recall)
write_recall_to_stats(E2_SEGMENTER_STATS, e2_recall)

### Evaluation of postprocessing order removal
* Computes and saves precision of order removal method for each edition.

In [6]:
# Manual annotation is done? 
# Evaluate
e1_removed_data = jh.read_items(E1_EVAL_ORDER)
e2_removed_data = jh.read_items(E2_EVAL_ORDER)

e1_removed_precision = precision(e1_removed_data, 'valid_removal')
e2_removed_precision = precision(e2_removed_data, 'valid_removal')

write_to_stats(e1_removed_precision, "Order removal precision", E1_ORDER_STATS)
write_to_stats(e2_removed_precision, "Order removal precision", E2_ORDER_STATS)

### Evaluation of postprocessing cross-reference linking
* Computes and saves precision of cross-reference linking method for each edition.

In [8]:
# Manual annotation done?
# Evaluate

e1_linked_data = jh.read_items(E1_TEST_LINKS)
e2_linked_data = jh.read_items(E2_TEST_LINKS)

e1_linked_precision = precision(e1_linked_data, 'valid_link')
e2_linked_precision = precision(e2_linked_data, 'valid_link')

write_to_stats(e1_linked_precision, "Linking precision", E1_STATS_LINKS)
write_to_stats(e2_linked_precision, "Linking precision", E2_STATS_LINKS)

In [4]:
e1_cross_samples = jh.read_items(E1_TEST_LINKS_JSON)
e2_cross_samples = jh.read_items(E2_TEST_LINKS_JSON)

e1_cross_ratio = sum([entry['is_cross_ref'] for entry in e1_cross_samples]) / len(e1_cross_samples)
e2_cross_ratio = sum([entry['is_cross_ref'] for entry in e2_cross_samples]) / len(e2_cross_samples)

e1_stats = jh.read_items(E1_STATS_LINKS_JSON)
e2_stats = jh.read_items(E2_STATS_LINKS_JSON)

e1_nr_entries = e1_stats[0]['Nr_articles']
e2_nr_entries = e2_stats[0]['Nr_articles']

e1_nr_cross_linked = e1_stats[0]['Nr_cross_ref_linked']
e2_nr_cross_linked = e2_stats[0]['Nr_cross_ref_linked']

e1_stats[0]['Recall'] = e1_nr_cross_linked / (e1_cross_ratio * e1_nr_entries)
e2_stats[0]['Recall'] = e2_nr_cross_linked / (e2_cross_ratio * e2_nr_entries)

jh.write_items(e1_stats, E1_STATS_LINKS_JSON)
jh.write_items(e2_stats, E2_STATS_LINKS_JSON)

